# How to set the Operating Point
This tutorial explains how to use the object InputCurrent and VarLoadCurrent to run a magnetic simulation on several operating points by setting Id/Iq or I0/Phi0.

The reference used to validate this tutorial is: Z. Yang, M. Krishnamurthy and I. P. Brown, "Electromagnetic and vibrational characteristic of IPM over full torque-speed range," 2013 International Electric Machines & Drives Conference, Chicago, IL, 2013, pp. 295-302, doi: 10.1109/IEMDC.2013.6556267.

## Machine and Simulation definition
This tutorial use the machine IPMSM_A (Prius 2004) defined in the "[How to define a machine](https://pyleecan.org/tuto_Machine.html)" tutorial. The magnetic module is the same as the symmetrical one from the tutorial "[How to define a simulation to call FEMM](https://pyleecan.org/tuto_Simulation_FEMM.html)".

In [ ]:
%matplotlib notebook

# Load the machine
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from os.path import join

IPMSM_A = load(join(DATA_DIR, "Machine", "IPMSM_A.json"))
IPMSM_A.plot()

In [ ]:
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.MagFEMM import MagFEMM
# Initialization of the Simulation
simu = Simu1(name="tuto_Id_Iq", machine=IPMSM_A)

# Definition of the magnetic simulation (FEMM with symmetry and sliding band)
simu.mag = MagFEMM(
    type_BH_stator=0,
    type_BH_rotor=0,
    is_periodicity_a=True,
    Kgeo_fineness=1,
)
# Run only Magnetic module
simu.elec = None
simu.force = None
simu.struct = None

## Defining an Operating point with Id/Iq
The starting point of the simulation is defined with InputCurrent. This object will create an "OutElec" object that correspond to the output of the Electrical module and the input of the Magnetic module. InputCurrent enables to define in several ways the starting point. The tutorial "[How to define a simulation to call FEMM](https://pyleecan.org/tuto_Simulation_FEMM.html)" introduces how to enforce any current by directly setting Is. When the current is sinusoidal, it can be defined with Id_ref and Iq_ref as follow:

In [ ]:
from pyleecan.Classes.InputCurrent import InputCurrent
from numpy import sqrt, exp,pi

# Definition of a sinusoidal current
simu.input = InputCurrent()
I0_rms = 250/sqrt(2) # Maximum current [Arms]
Phi0 = 140*pi/180  # Maximum Torque Per Amp
Id_ref = (I0_rms*exp(1j*Phi0)).real
Iq_ref = (I0_rms*exp(1j*Phi0)).imag
simu.input.Id_ref = Id_ref # [Arms] 
simu.input.Iq_ref = Iq_ref # [Arms] 

(Id_ref,Iq_ref)

The discretization of the current and for the magnetic computation can be set with time and angle (as in "[How to define a simulation to call FEMM](https://pyleecan.org/tuto_Simulation_FEMM.html)" tutorial) or by setting the following parameters:

In [ ]:
simu.input.Nt_tot = 10 # Number of time step
simu.input.Na_tot = 2048 # Spatial discretization
simu.input.N0 = 2000 # Rotor speed [rpm]

When Nt_tot is defined, the time vector is automatically set to:

    linspace(0, 60 / N0 * Nrev, Nt_tot)

With Nrev the number of revolution of the rotor (1 by default)

When Na_tot is defined, the angle vector is automatically set to:

    linspace(0, 2*pi, Na_tot)

The input is now fully defined, the simulation can now be run:

In [ ]:
out = simu.run()
# Plot the flux
out.plot_2D_Data("mag.B", "angle")
# Plot the current
out.plot_2D_Data("elec.Is", "time", "phase")

The Operating Point can also be defined with I0 / Phi0 with:

In [ ]:
from numpy import pi

simu.input.set_Id_Iq(I0=I0_rms, Phi0=Phi0)
print("Id: "+str(simu.input.Id_ref))
print("Iq: "+str(simu.input.Iq_ref))

## Iterating on several Operating Point
Each pyleecan simulation is assumed to be quasi-static and run on a single operating point (fixed speed). To run a simulation on several operating point two steps are needed: First define a simulation that run correctly on a single operating point (like the one defined above), then define a VarLoadCurrent object. 

The VarLoadCurrent object is defined with a matrix with each line corresponding to an operating point and the column are either (N0, Id, Iq) if type_OP_matrix==1 or (N0, I0, Phi0) if type_OP_matrix==0. The following VarLoadCurrent object will run 36 time the previous simulation by changing the value of Phi0. 

A fourth column can be added by setting is_torque=True. It enables to define the reference torque for the Operating Point. The reference is stored in output.elec.Tem_av_ref, the real computed torque is stored in output.mag.Tem_av



Reference torque and current angle vector are:

In [ ]:
from numpy import linspace, array, pi
Tem_av_ref = array([79, 125, 160, 192, 237, 281, 319, 343, 353, 332, 266, 164, 22]) # Yang et al, 2013
Phi0_ref = linspace(60 * pi / 180, 180 * pi / 180, Tem_av_ref.size)

In [ ]:
from pyleecan.Classes.VarLoadCurrent import VarLoadCurrent
from numpy import zeros, ones, linspace, array, sqrt, arange

varload = VarLoadCurrent(is_torque=True, ref_simu_index=0)
varload.type_OP_matrix = 0 # Matrix N0, I0, Phi0

# Choose which operating points to run
step = 2 # step=1 to do all OP
         # step=2 to do 1 OP out of 2 (fastest)    
I_simu = arange(0,Tem_av_ref.size, step) 
N_simu = I_simu.size

# Creating the Operating point matrix
OP_matrix = zeros((N_simu,4))

# Set N0 = 2000 [rpm] for all simulation
OP_matrix[:,0] = 2000 * ones((N_simu))

# Set I0 = 250 / sqrt(2) [A] (RMS) for all simulation
OP_matrix[:,1] = I0_rms * ones((N_simu)) 

# Set Phi0 from 60° to 180°
OP_matrix[:,2] = Phi0_ref[I_simu]

# Set reference torque from Yang et al, 2013
OP_matrix[:,3] = Tem_av_ref[I_simu]

varload.OP_matrix = OP_matrix
print(OP_matrix)

The original simulation will be duplicated N_simu times with the value of InputCurrent updated according to the matrix.

In [ ]:
simu2 = simu.copy()
simu2.var_simu = varload.copy()
simu2.input.Nt_tot = 5
Xout = simu2.run()

Pyleecan will automatically extract some values from each simulation. These values are all gathered in the xoutput_dict:

In [ ]:
print("Values available in XOutput:")
print(Xout.xoutput_dict.keys())

print("\nI0 for each simulation:")
print(Xout["I0"].result)
print("\nPhi0 for each simulation:")
print(Xout["Phi0"].result)

Any parameter in the XOutput can be plot as a function of any other

In [ ]:
fig = Xout.plot_multi("Phi0", "Tem_av")
fig = Xout.plot_multi("Id", "Iq")

Finally, the computed average torque can be compared to the one in the publication from Yang et al (data has been extracted from their graph using [Engauge Digitizer](http://markummitchell.github.io/engauge-digitizer/). Note that the generic plot function `plot_2D` has been used here.

In [ ]:
from pyleecan.Functions.Plot.plot_2D import plot_2D
from pyleecan.definitions import config_dict
from numpy import array

curve_colors = config_dict["PLOT"]["COLOR_DICT"]["CURVE_COLORS"]

plot_2D(
    array([x*180/pi for x in Xout.xoutput_dict["Phi0"].result]),
    [Xout.xoutput_dict["Tem_av"].result, Xout.xoutput_dict["Tem_av_ref"].result],
    color_list=curve_colors,
    legend_list=["Pyleecan", "Yang et al, 2013"],
    xlabel="Current angle [°]",
    ylabel="Electrical torque [N.m]",
    title="Electrical torque vs current angle"
)